In [1]:
from keras.models import load_model
import tempfile
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Layer
from tensorflow import keras
from keras_unet_collection import models
import keras_unet

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [2]:
#define custom objects
def jaccard_distance(y_true, y_pred, smooth=100):
        intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
        sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
        jac = (intersection + smooth) / (sum_ - intersection + smooth)
        return (1 - jac) * smooth

def dice_coef(y_true, y_pred):
    smooth = 1.0
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (
                K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(self, y_true, y_pred):
    loss = 1 - self._dice_coef(y_true, y_pred)
    return loss

class GELU(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(GELU, self).__init__(**kwargs)
    
    def call(self, inputs):
        return keras.activations.gelu(inputs)

In [3]:
import h5py
import numpy as np
from sklearn.decomposition import PCA

model = keras.models.load_model(r"C:\Users\UAB\Segmentation - Main1_CK\Human Model\Keras\runet_kid_best_train.h5", custom_objects={
                       'jaccard_distance': jaccard_distance,
                       'dice_coef_loss': dice_coef_loss,
                       'dice_coef': dice_coef}) #load the weights




In [22]:
weights = model.get_weights()
# Flatten the weights into a 2D matrix
flattened_weights = np.concatenate([w.flatten() for w in weights])
flattened_weights = flattened_weights.reshape(-1, 1)

# Perform PCA on the weight matrix
pca = PCA()
pca.fit(flattened_weights)

# Select a subset of the principal components based on the desired compression ratio
compression_ratio = 0.5  # Example compression ratio of 50%
num_components = int(len(flattened_weights) * compression_ratio)
selected_components = pca.components_[:num_components]

# Reconstruct the compressed weights using the selected principal components
compressed_weights = np.dot(selected_components, pca.transform(flattened_weights).T).T

# Update the model with the compressed weights
start_index = 0
for i in range(len(weights)):
    shape = weights[i].shape
    size = np.prod(shape)
    end_index = start_index + size
    weights[i] = compressed_weights[start_index:end_index].reshape(shape)
    start_index = end_index

model.set_weights(weights)

In [25]:
import numpy as np

# Calculate the magnitude of each weight
weight_magnitudes = []
for layer in model.layers:
    if hasattr(layer, 'get_weights'):
        weights = layer.get_weights()
        for weight in weights:
            weight_magnitudes.append(np.abs(weight).flatten())

# Concatenate the weight magnitudes into a single array
weight_magnitudes = np.concatenate(weight_magnitudes)

# Normalize the magnitudes
weight_magnitudes /= np.sum(weight_magnitudes)

In [26]:
pruning_ratio = 0.5  # Example pruning ratio of 50%


In [27]:
sorted_indices = np.argsort(weight_magnitudes)[::-1]
threshold_index = int(len(sorted_indices) * pruning_ratio)
threshold = weight_magnitudes[sorted_indices[threshold_index]]

In [28]:
from tensorflow_model_optimization.sparsity import keras as sparsity

# Define the pruning schedule
pruning_schedule = sparsity.ConstantSparsity(target_sparsity=pruning_ratio, begin_step=0, end_step=-1, frequency=100)

# Apply pruning to the model
pruned_model = sparsity.prune_low_magnitude(model, pruning_schedule=pruning_schedule)

# Prune the model by applying the threshold to each weight
for layer in pruned_model.layers:
    if hasattr(layer, 'get_prunable_weights'):
        prunable_weights = layer.get_prunable_weights()
        pruned_weights = []
        for weight in prunable_weights:
            pruned_weight = np.where(np.abs(weight) < threshold, 0.0, weight)
            pruned_weights.append(pruned_weight)
        layer.set_weights(pruned_weights)


In [29]:
final_model = sparsity.strip_pruning(pruned_model)
